In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("../data/raw/train.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id_old       8000 non-null   int64  
 1   Id           8000 non-null   int64  
 2   GP           8000 non-null   int64  
 3   MIN          8000 non-null   float64
 4   PTS          8000 non-null   float64
 5   FGM          8000 non-null   float64
 6   FGA          8000 non-null   float64
 7   FG%          8000 non-null   float64
 8   3P Made      8000 non-null   float64
 9   3PA          8000 non-null   float64
 10  3P%          8000 non-null   float64
 11  FTM          8000 non-null   float64
 12  FTA          8000 non-null   float64
 13  FT%          8000 non-null   float64
 14  OREB         8000 non-null   float64
 15  DREB         8000 non-null   float64
 16  REB          8000 non-null   float64
 17  AST          8000 non-null   float64
 18  STL          8000 non-null   float64
 19  BLK   

No non-null and all values are numeric

In [4]:
df.describe(include='all')

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
count,8000.000000,8000.00000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,6856.971000,7798.50000,62.777875,18.576662,7.267088,2.807037,6.231212,44.608900,0.264525,0.816562,19.583700,1.392525,1.947788,71.365825,1.077838,2.168500,3.245300,1.624513,0.648687,0.245212,1.257763,0.833625
std,3977.447579,2309.54541,17.118774,8.935263,4.318732,1.693373,3.584559,6.155453,0.384093,1.060964,16.003155,0.926153,1.252352,10.430447,0.785670,1.392224,2.085154,1.355986,0.407626,0.821037,0.723270,0.372440
min,4.000000,3799.00000,-8.000000,2.900000,0.800000,0.300000,0.800000,21.300000,-1.100000,-3.100000,-38.500000,0.000000,0.000000,-13.300000,0.000000,0.200000,0.300000,0.000000,0.000000,-17.900000,0.100000,0.000000
25%,3413.750000,5798.75000,51.000000,12.000000,4.100000,1.600000,3.600000,40.400000,0.000000,0.100000,8.400000,0.700000,1.000000,65.000000,0.500000,1.100000,1.700000,0.700000,0.300000,0.100000,0.700000,1.000000
50%,6787.500000,7798.50000,63.000000,16.800000,6.300000,2.400000,5.400000,44.400000,0.300000,0.800000,19.500000,1.200000,1.700000,71.400000,0.900000,1.900000,2.800000,1.300000,0.600000,0.200000,1.100000,1.000000
75%,10299.250000,9798.25000,74.000000,23.500000,9.500000,3.700000,8.100000,48.700000,0.500000,1.500000,30.600000,1.900000,2.600000,77.500000,1.500000,2.900000,4.300000,2.200000,0.900000,0.400000,1.600000,1.000000
max,13798.000000,11798.00000,123.000000,73.800000,34.200000,13.100000,28.900000,67.200000,1.700000,4.700000,82.100000,8.100000,11.100000,168.900000,5.500000,11.000000,15.900000,12.800000,3.600000,18.900000,5.300000,1.000000


## Data Quality Concerns
- GP, 3PM, 3PA, 3p%, FT%, BLK have negative value as minimum - **Convert to absolute value**
- There are potential outliers in the value between 75% mark and MAX is huge jump across all features
- Check if all made value is smaller than attempt value
- All % values are bit off and not close to Made/attempt - **Consider dropping these fields or recreate them**
- BLK has outliers - **Fix it or drop these**
- 

In [5]:
df["TARGET_5Yrs"].value_counts()

1    6669
0    1331
Name: TARGET_5Yrs, dtype: int64

## Imbalanced Data
The ground truth is quite unbalanced. Need to keep in mind in model selection and evaluation approach

In [6]:
df_min = df.loc[df['TARGET_5Yrs'] == 0]
df_min

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
12,13144,3811,41,9.8,2.3,0.9,2.3,37.9,-0.4,-0.6,-4.7,0.9,1.3,64.7,1.5,1.9,3.5,0.2,0.3,0.4,0.4,0
23,8590,3822,71,10.1,4.0,1.4,3.0,47.7,0.4,0.7,31.4,1.3,1.6,84.3,0.3,0.5,0.8,1.8,0.5,-0.2,1.6,0
25,3556,3824,34,10.5,3.3,1.2,2.7,43.6,0.0,0.0,23.4,1.2,1.7,64.9,1.5,2.4,3.8,0.5,0.2,0.6,0.6,0
34,709,3833,58,12.5,3.3,1.2,3.1,40.9,0.2,0.7,17.3,0.9,1.5,55.1,0.2,0.7,1.0,2.5,0.4,0.1,1.0,0
44,12340,3843,106,45.0,20.0,7.5,14.7,52.6,0.4,1.2,21.5,4.6,6.4,71.4,2.6,6.0,9.3,3.5,1.7,0.5,2.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7960,13785,11759,49,17.2,6.1,2.4,6.1,39.1,0.7,1.9,37.1,0.7,0.7,91.7,0.2,1.1,1.4,2.1,0.3,-1.4,0.7,0
7966,11565,11765,26,6.3,1.7,0.6,1.7,38.2,0.2,0.6,2.4,0.3,0.4,71.5,0.3,0.7,1.0,0.2,0.3,0.0,0.4,0
7972,13636,11771,63,4.5,1.7,0.6,1.3,47.4,-0.5,-1.1,1.7,0.7,1.2,62.6,0.9,1.1,2.2,0.0,0.1,0.3,0.4,0
7989,3145,11788,57,27.8,13.9,5.0,10.7,46.6,0.4,1.2,12.3,3.7,5.1,72.9,1.0,2.7,3.8,2.2,0.8,0.1,2.5,0


In [14]:
from sklearn.utils import resample
df_min = resample(df_min, replace=True, n_samples=6669, random_state=123)
len(df_min.index)

6669

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,2857,8103,42,8.0,2.7,1.0,2.9,34.9,0.5,1.5,22.0,0.3,0.5,64.7,0.3,1.2,1.4,0.5,0.2,0.2,0.6,1
1,11478,7943,74,21.2,3.9,1.5,4.1,39.8,0.5,1.5,33.1,0.5,0.8,62.2,0.5,2.0,2.6,2.3,0.9,0.5,1.3,1
2,10740,5420,73,14.8,6.1,2.4,5.4,43.2,0.2,0.5,14.0,1.4,2.4,55.2,0.9,1.3,2.0,1.2,0.4,0.4,1.0,1
3,4032,8687,38,9.3,2.5,0.9,2.6,36.2,0.7,1.7,25.1,0.4,0.5,75.4,0.1,0.7,0.8,2.0,0.6,-1.6,0.6,1
4,7503,10083,47,18.2,5.4,2.0,4.0,49.2,-0.3,-1.0,4.7,1.6,2.3,74.2,1.8,3.5,5.2,1.0,0.7,0.3,1.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13333,6126,10647,68,23.5,13.0,5.4,9.9,55.3,0.1,0.1,20.9,2.2,3.0,69.0,2.4,2.6,4.9,1.3,0.7,0.3,1.8,0
13334,7088,4573,39,6.7,2.1,0.7,1.7,40.9,0.4,1.0,28.7,0.6,0.7,79.6,0.8,0.6,1.3,0.0,0.1,0.2,0.3,0
13335,5022,11201,63,11.0,5.7,2.3,5.4,42.0,-0.1,-0.2,5.6,1.3,1.9,71.5,1.5,1.3,2.5,1.3,0.4,0.2,0.9,0
13336,1373,9473,64,16.7,5.5,2.4,4.4,54.2,0.0,0.0,13.0,0.7,1.0,72.3,1.5,1.7,3.2,0.5,0.5,0.3,0.9,0
